In [1]:
from keras.layers import Input, Dense,Flatten, Conv2D, MaxPooling2D
from keras.models import Model
import tensorflow as tf
import os
import numpy as np
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from keras_flops import get_flops

shape_x = 224
shape_y = 224

X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
Y_train=np.random.randint(0,10,size=[60,])
X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
Y_test=np.random.randint(0,10,size=[60,])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
def VGG19(input_shape):
    
    """
    

    Arguments:
    input_shape -- shape of the images of the dataset
    
    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    #Block 1
    X = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(X_input)
    X = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(X)
    #Block 2
    X = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(X)
    X = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(X)
    #Block 3
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(X)
    #Block 4
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(X)
    #Block 5
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(X)
    
    
    X = Flatten(name='flatten')(X)
    X = Dense(4096, activation='relu', name='fc1')(X)
    X = Dense(4096, activation='relu', name='fc2')(X)    
    
    # 输出层
    outputs = Dense(10, activation='softmax')(fc2)
    
    model = Model(inputs=X_input, outputs=outputs)
    
    return model

input_shape = (shape_x, shape_y, 1)
model = VGG19(input_shape)
model.summary()
Total_params = round(model.count_params()/10 ** 6,2)
FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
print(Total_params)
print(FLOPs)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, Y_train, epochs=1)
# model.get_weights()[0].dtype
# quantizer = vitis_quantize.VitisQuantizer(model)
# quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
# quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
# quantized_model.save('tf2_mnist_classifier_quantized.h5')
# !vai_c_tensorflow2 \
# --model ./tf2_mnist_classifier_quantized.h5 \
# --arch ./arch.json \
# --output_dir ./xmodel_vgg19 \
# --net_name deploy
# os.rename("./xmodel_vgg19/deploy.xmodel","./xmodel_vgg19/vgg19_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))

2022-03-30 05:10:55.644085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-03-30 05:10:55.644115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-30 05:10:56.748631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-03-30 05:10:56.748662: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

2022-03-30 05:10:57.559178: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-03-30 05:10:57.559318: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-03-30 05:10:57.579462: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number

In [1]:
import os
import numpy as np
import random

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from itertools import product

def VGG19(input_shape):
    
    """
    Arguments:
    input_shape -- shape of the images of the dataset
    
    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    #Block 1
    conv1_kernel_num = random.randrange(32,64,2)
    X = Conv2D(conv1_kernel_num, (3, 3), activation='relu', padding='same', name='block1_conv1')(X_input)
    conv2_kernel_num = random.randrange(conv1_kernel_num,64,2)
    X = Conv2D(conv2_kernel_num, (3, 3), activation='relu', padding='same', name='block1_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(X)
    #Block 2
    conv3_kernel_num = random.randrange(conv2_kernel_num,128,2)
    X = Conv2D(conv3_kernel_num, (3, 3), activation='relu', padding='same', name='block2_conv1')(X)
    conv4_kernel_num = random.randrange(conv3_kernel_num,128,2)
    X = Conv2D(conv4_kernel_num, (3, 3), activation='relu', padding='same', name='block2_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(X)
    #Block 3
    conv5_kernel_num = random.randrange(conv4_kernel_num,256,2)
    X = Conv2D(conv5_kernel_num, (3, 3), activation='relu', padding='same', name='block3_conv1')(X)
    conv6_kernel_num = random.randrange(conv5_kernel_num,256,2)
    X = Conv2D(conv6_kernel_num, (3, 3), activation='relu', padding='same', name='block3_conv2')(X)
    conv7_kernel_num = random.randrange(conv6_kernel_num,256,2)
    X = Conv2D(conv7_kernel_num, (3, 3), activation='relu', padding='same', name='block3_conv3')(X)
    conv8_kernel_num = random.randrange(conv7_kernel_num,256,2)
    X = Conv2D(conv8_kernel_num, (3, 3), activation='relu', padding='same', name='block3_conv4')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(X)
    #Block 4
    conv9_kernel_num = random.randrange(conv8_kernel_num,512,2)
    X = Conv2D(conv9_kernel_num, (3, 3), activation='relu', padding='same', name='block4_conv1')(X)
    conv10_kernel_num = random.randrange(conv9_kernel_num,512,2)
    X = Conv2D(conv10_kernel_num, (3, 3), activation='relu', padding='same', name='block4_conv2')(X)
    conv11_kernel_num = random.randrange(conv10_kernel_num,512,2)
    X = Conv2D(conv11_kernel_num, (3, 3), activation='relu', padding='same', name='block4_conv3')(X)
    conv12_kernel_num = random.randrange(conv11_kernel_num,512,2)
    X = Conv2D(conv12_kernel_num, (3, 3), activation='relu', padding='same', name='block4_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(X)
    #Block 5
    conv13_kernel_num = random.randrange(conv12_kernel_num,512,2)
    X = Conv2D(conv13_kernel_num, (3, 3), activation='relu', padding='same', name='block5_conv1')(X)
    conv14_kernel_num = random.randrange(conv13_kernel_num,512,2)
    X = Conv2D(conv14_kernel_num, (3, 3), activation='relu', padding='same', name='block5_conv2')(X)
    conv15_kernel_num = random.randrange(conv14_kernel_num,512,2)
    X = Conv2D(conv15_kernel_num, (3, 3), activation='relu', padding='same', name='block5_conv3')(X)
    conv16_kernel_num = random.randrange(conv15_kernel_num,512,2)
    X = Conv2D(conv16_kernel_num, (3, 3), activation='relu', padding='same', name='block5_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(X)
    
    
    X = Flatten(name='flatten')(X)
    Dense1_neuron_num = random.randrange(1072,3072,32)
    X = Dense(Dense1_neuron_num, activation='relu', name='fc1')(X)
    Dense2_neuron_num = random.randrange(1072,Dense1_neuron_num+1,32)
    X = Dense(Dense2_neuron_num, activation='relu', name='fc2')(X)    
    # 输出层
    outputs = Dense(10, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=outputs)
    
    return model

shape_range =range(32,320,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

    input_shape = (shape_x, shape_y, 1)
    for step in range(1,11):
        model = VGG19(input_shape)
        model.summary()
        model.save('./vgg19_h5/vgg19_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))


#         model.compile(optimizer='adam',
#                   loss="sparse_categorical_crossentropy", 
#                   metrics=['accuracy'])
#         history = model.fit(X_train, Y_train, epochs=1)
#         model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg19 \
                --net_name deploy
        os.rename("./xmodel_vgg19/deploy.xmodel","./xmodel_vgg19/xmodel_vgg19_{shape_x}_{shape_y}_{step}.xmodel".format(shape_x=shape_x,shape_y=shape_y,step=step))

2022-04-13 04:06:47.875180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-13 04:06:47.875210: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 56)        560       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 60)        30300     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 60)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 72)        38952     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 110)       71390     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 110)         0     

2022-04-13 04:06:49.466107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-13 04:06:49.466142: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-13 04:06:49.466159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-04-13 04:06:49.466298: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 410ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2022-04-13 04:07:01.198363: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


...

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 528ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 8s 8s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[INFO] generate xmodel     :100%|█| 76/76 [00:03<00:00, 21.77it/s]              
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B2304_MAX_BG2
[UNILOG][INFO] Graph name: model_69, with op num: 146
[UNILOG][INFO] Begin to compile...
[UNILOG][WARNING] xir::Op{name = quant_fc1(TransferMatMulToConv2d), type = conv2d-fix} has been assigned to CPU: [Weights(2096, 7, 7, 510) is too large to be loaded into parameter buffer. "kernel_h * kernel_w * input_channel" is supposed to be less equal than 24576].
[UNILOG][WARNING] xir::Op{name = quant_fc1_const_const(reshaped), type = const-fix} has been assigned to CPU: [Has no fanout or at least one fanout is out of DPU subgraph.].
[UNILOG][WARNING] xir::Op{name = quant_fc1_bias, type = const-fix} has been assigned to CPU: [Has no fanout or at least one fanout is out of DPU subgraph.].
[UNILOG][INFO] Total device subgraph number 5, DPU sub

Model: "model_71"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_72 (InputLayer)        [(None, 256, 256, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 38)      380       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 48)      16464     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 48)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 110)     47630     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 116)     114956    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 116)       0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 6s 720ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

KeyboardInterrupt: 